# Create Guild

In [1]:
import os, sys
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM

In [7]:
import engine.models as GM
import engine.logic.resource as RS

In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Starting Guild

In [9]:
users = DB.GetTableDictionary('members', 'User')
PD.DataFrame(users).drop(['password'], axis=1, errors='ignore')

,id,last_login,is_superuser,unique_id,email,user_name,verified,active,date_joined
0,1,None,True,E80-F16-216,admin@thieves-guild.com,Admin,True,True,2023-08-07 00:54:17.471207+00:00
1,2,None,False,8B5-076-999,testor1@thieves-guild.com,Testor1,True,True,2023-08-07 00:54:17.809681+00:00


In [10]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [11]:
DB.DeleteTable('engine', 'Guild')

table deleted


In [12]:
newGuild = RS.CreateStartingGuild(userMd, '123')

PD.DataFrame([newGuild.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,KeepLevel,TotalPower,VaultGold,VaultGems,Selected,LastPlayed,CreateDate
0,1,123,1,0,0,0,True,2023-08-12 21:47:45.639127+00:00,2023-08-12 21:47:45.639127+00:00


In [13]:
newThief = RS.AppendStartingThief(newGuild, 'Burglar', 1)
RS.AttachStartingWargear(newThief)
# newThief = RS.AppendStartingThief(newGuild, 'Scoundrel', 1)
# RS.AttachStartingWargear(newThief)
# newThief = RS.AppendStartingThief(newGuild, 'Ruffian', 1)
# RS.AttachStartingWargear(newThief)

thiefOb = GM.ThiefInGuild.objects.filter(GuildFK=newGuild)
for th in thiefOb:
    RS.SetThiefTotals(th)

In [14]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=newGuild).values()
PD.DataFrame(thieves).drop(['id', 
        'Name', 'Experience', 'Position',
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
#         'Attack', 'Damage', 'Defense',
        'Traverse', 'Sabotage', 'Perceive', 
        ], axis=1, errors='ignore')

,GuildFK_id,Class,Stars,BasePower,Level,Power,Agility,Cunning,Might,Endurance,Health,Attack,Damage,Defense,Wounds,Cooldown
0,20,Burglar,1,30,1,60,5,0,0,0,70,5,11,13,None,None
1,20,Scoundrel,1,30,1,60,0,5,0,0,70,2,15,12,None,None
2,20,Ruffian,1,30,1,60,0,0,5,0,70,3,10,16,None,None
3,20,Burglar,1,30,1,60,5,0,0,0,70,5,11,13,None,None


In [15]:
guildItems = GM.ItemInGuild.objects.filter(GuildFK=newGuild).values()
PD.DataFrame(guildItems).drop(['id'], axis=1, errors='ignore')

,GuildFK_id,ThiefFK_id,Name,Level,Slot,Power,Requirement,Trait,Attack,Damage,Defense,Skill,Magic
0,20,65.0,Blackthorn,1,weapon,15,Burglar,agi 1,NaN,11.0,NaN,None,None
1,20,65.0,Brigandine,1,armor,15,Burglar,agi 1,NaN,NaN,3.0,None,None
2,20,66.0,Twin Daggers,1,weapon,15,Scoundrel,cun 1,2.0,10.0,NaN,None,None
3,20,66.0,Vestment,1,armor,15,Scoundrel,cun 1,NaN,NaN,2.0,None,None
4,20,67.0,Sword & Shield,1,weapon,15,Ruffian,mig 1,3.0,10.0,NaN,None,None
5,20,67.0,Chain Maille,1,armor,15,Ruffian,mig 1,NaN,NaN,1.0,None,None
6,20,NaN,Helmet,1,head,10,None,None,NaN,NaN,NaN,per 5,None
7,20,NaN,Gloves,1,hands,10,None,None,NaN,NaN,NaN,sab 5,None
8,20,NaN,Boots,1,feet,10,None,None,NaN,NaN,NaN,tra 5,None
9,20,68.0,Blackthorn,1,weapon,15,Burglar,agi 1,NaN,11.0,NaN,None,None
